##### Copyright 2023 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import


In [ ]:
!pip install -U -q google-generativeai

In [ ]:
# import necessary modules.
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown

## Set the API key

Add your API_KEY to the secrets manager in the left pannel "🔑".

In [ ]:
from google.colab import userdata

API_KEY=userdata.get('API_KEY')

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

### Parse the arguments

In [ ]:
model = 'gemini-1.0-pro' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIifSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjoiKipQZXJmaWwgTGlww61kaWNvKipcblxuKiBDb2xlc3Rlcm9sIExETDogOTEsMCBtZy9kbCAow5NwdGltbylcbiogQ29sZXN0ZXJvbCBWTERMOiAxNyw2IG1nL2RsIChFbnRyZSBub3JtYWwgeSBhbHRvKVxuKiBUcmlnbGljw6lyaWRvczogODgsMCBtZy9kbCAoRW50cmUgbm9ybWFsIHkgYWx0bylcbiogQ29sZXN0ZXJvbCBIREw6IDQxLDAgbWcvZGwgKFNpbiByaWVzZ28pXG4qIENvbGVzdGVyb2wgdG90YWw6IDE0MywwIG1nL2RsIChOb3JtYWwpXG5cbioqQmlvcXXDrW1pY2EqKlxuXG4qIENyZWF0aW5pbmEgZW4gc3Vlcm86IDEsMDQgbWcvZGwgKE5vcm1hbClcbiogTml0csOzZ2VubyB1cmVpY28gKEJVTik6IDEwLDEgbWcvZGwgKE5vcm1hbClcbiogVXJlYTogMjEsNiBtZy9kbCAoTm9ybWFsKVxuKiBIZW1vZ2xvYmluYSBnbGljb3NpbGFkYSBIYkExYzogNSwyICUgKE5vcm1hbClcbiogR2x1Y2VtaWE6IDg5LDAgbWcvZGwgKE5vcm1hbClcblxuKipIZW1hdG9sb2fDrWEqKlxuXG4qKkN1YWRybyBoZW3DoXRpY28gYWx0YSB0ZWNub2xvZ8OtYSoqXG5cbiogUkRXOiAxMywzMCAlIChOb3JtYWwpXG4qIFJlY3VlbnRvIHRvdGFsIGRlIHBsYXF1ZXRhczogMjYxIHgxMF4zL3VMIChOb3JtYWwpXG4qIE1DSEM6IDMzLDkgZ3IvZGwgKE5vcm1hbClcbiogTUNIOiAzMCwwIHBnIChOb3JtYWwpXG4qIE1DVjogODgsNiBmbCAoTm9ybWFsKVxuKiBIZW1hdG9jcml0bzogNDMsNyAlIChOb3JtYWwpXG4qIEhlbW9nbG9iaW5hOiAxNCw4MCBnci9kbCAoTm9ybWFsKVxuKiBSZWN1ZW50byBkZSBoZW1hdMOtZXM6IDQsOTMgWDEwKjYvbW3CsyAoTm9ybWFsKVxuKiBDw6lsdWxhcyBpbm1hZHVyYXMgJTogMCw1MCAlIChOb3JtYWwpXG4qIEPDqWx1bGFzIGlubWFkdXJhczogMCwwMzAgeDEwXjMvdUwgKE5vcm1hbClcbiogTm9ybW9ibGFzdG9zICU6IDAsMDAgJSAoTm9ybWFsKVxuKiBOb3Jtb2JsYXN0b3MgQUJTOiAwLDAwIHgxMF4zL3VMIChOb3JtYWwpXG4qIFJlY3VlbnRvIGRlIGxldWNvY2l0b3M6IDYsMjkgWDEwwrMvbW3CsyAoTm9ybWFsKVxuXG4qKlZlbG9jaWRhZCBkZSBzZWRpbWVudGFjacOzbiBnbG9idWxhciAoVlNHKSoqXG5cbiogMywwIG1tL2hvcmEgKE5vcm1hbClcblxuKipEaWZlcmVuY2lhbCBkZSBsZXVjb2NpdG9zKipcblxuKiBCYXPDs2ZpbG9zIEFiczogMCwwNSB4MTBeMy91TCAoTm9ybWFsKVxuKiBFb3NpbsOzZmlsb3MgQWJzOiAwLDEzIHgxMF4zL3VMIChOb3JtYWwpXG4qIE1vbm9jaXRvcyBBYnM6IDAsNDkgeDEwXjMvdUwgKE5vcm1hbClcbiogTGluZm9jaXRvcyBBYnM6IDIsMjggeDEwXjMvdUwgKE5vcm1hbClcbiogTmV1dHLDs2ZpbG9zIEFiczogMywzMSB4MTBeMy91TCAoTm9ybWFsKVxuKiBCYXPDs2ZpbG9zICU6IDAsODAgJSAoTm9ybWFsKVxuKiBFb3NpbsOzZmlsb3MgJTogMiwxMCAlIChOb3JtYWwpXG4qIE1vbm9jaXRvcyAlOiA3LDgwICUgKE5vcm1hbClcbiogTGluZm9jaXRvcyAlOiAzNiwyMCAlIChOb3JtYWwpXG4qIE5ldXRyw7NmaWxvcyAlOiA1Miw2MCAlIChOb3JtYWwpXG4qIE1QVjogMTAsMCBmbCAoTm9ybWFsKVxuXG4qKkVuZG9jcmlub2xvZ8OtYSoqXG5cbiogVGlyb3hpbmEgbGlicmUgKFQ0IGxpYnJlKTogMSw1NiBuZy9kbCAoTm9ybWFsKVxuKiBUU0ggaG9ybW9uYSBlc3RpbXVsYW50ZSBkZSB0aXJvaWRlczogMiw2NCB1VUkvbWwgKE5vcm1hbClcblxuKipVcm9hbsOhbGlzaXMqKlxuXG4qKlBhcmNpYWwgZGUgb3JpbmEgYXV0b21hdGl6YWRvKipcblxuKiBMZXVjb2NpdG9zL2VzdGVhcmFzYTogMjUgL3VMXG4qIHBIOiA2LDBcbiogRGVuc2lkYWQ6IDEwMjJcbiogQXNwZWN0bzogVHVyYmlvICtcbiogQ29sb3I6IEFtYXJpbGxvXG5cbioqU2VkaW1lbnRvIHVyaW5hcmlvIGF1dG9tYXRpemFkbyByZXBvcnRlIGVuIGNlbGwvdUwqKlxuXG4qIENlbHVsYXMgZXBpdGVsaWFsZXMgZXNjYW1vc2FzOiBFc2Nhc2FzIENlbGxzL8K1TFxuKiBDZWx1bGFzIGVwaXRlbGlhbGVzIG5vIGVzY2Ftb3NhczogRXNjYXNhcyBDZWxscy/CtUxcbiogTGV1Y29jaXRvczogNSw5NCBDZWxscy/CtUxcbiogRXJpdHJvY2l0b3M6IDwxLDAwIENlbGxzL8K1TFxuXG4qKkV4YW1lbiBjdWFsaXRhdGl2byB1cmluYXJpbyoqXG5cbiogRXJpdHJvY2l0b3M6IE5lZ2F0aXZvIC91TFxuKiBCaWxpcnJ1YmluYTogTmVnYXRpdm8gbWcvZGxcbiogVXJvYmlsaW7Ds2dlbm86IE5vcm1hbCBtZy9kbFxuKiBDZXRvbmFzOiBOZWdhdGl2byBtZy9kbFxuKiBHbHVjb3NhOiBOb3JtYWwgbWcvZGxcbiogUHJvdGXDrW5hczogTmVnYXRpdm8gbWcvZGxcbiogTml0cml0b3M6IE5lZ2F0aXZvIn0seyJyb2xlIjoidXNlciIsInBhcnRzIjoiZXhwbGljYW1lICBsb3MgcmVzdWx0YWRvcyBzaSBzb24gYnVlbm9zIG8gbWFsb3MgcGFyYSB1bmEgcGVyc29uYSBuYXR1cmFsIHNpbiBjb25vY2ltaWVudG9zIGRlIG1lZGljaW5hIn0seyJyb2xlIjoibW9kZWwiLCJwYXJ0cyI6IioqQnVlbm9zIHJlc3VsdGFkb3M6KipcblxuKiAqKlBlcmZpbCBsaXDDrWRpY286KipcbiAgICAqIENvbGVzdGVyb2wgTERMOiDDs3B0aW1vXG4gICAgKiBDb2xlc3Rlcm9sIEhETDogc2luIHJpZXNnb1xuICAgICogQ29sZXN0ZXJvbCB0b3RhbDogbm9ybWFsXG4qICoqQmlvcXXDrW1pY2E6KipcbiAgICAqIENyZWF0aW5pbmEgZW4gc3Vlcm8sIG5pdHLDs2dlbm8gdXJlaWNvIChCVU4pLCB1cmVhLCBoZW1vZ2xvYmluYSBnbGljb3NpbGFkYSBIYkExYyB5IGdsdWNlbWlhOiB0b2RvcyBub3JtYWxlc1xuKiAqKkhlbWF0b2xvZ8OtYToqKlxuICAgICogQ3VhZHJvIGhlbcOhdGljbyBhbHRhIHRlY25vbG9nw61hOiB0b2RvcyBsb3MgdmFsb3JlcyBkZW50cm8gZGVsIHJhbmdvIG5vcm1hbFxuICAgICogVmVsb2NpZGFkIGRlIHNlZGltZW50YWNpw7NuIGdsb2J1bGFyIChWU0cpOiBub3JtYWxcbiogKipFbmRvY3Jpbm9sb2fDrWE6KipcbiAgICAqIFRpcm94aW5hIGxpYnJlIChUNCBsaWJyZSkgeSBUU0ggaG9ybW9uYSBlc3RpbXVsYW50ZSBkZSB0aXJvaWRlczogYW1ib3Mgbm9ybWFsZXNcblxuKipSZXN1bHRhZG9zIGxpZ2VyYW1lbnRlIGVsZXZhZG9zOioqXG5cbiogKipQZXJmaWwgbGlww61kaWNvOioqXG4gICAgKiBDb2xlc3Rlcm9sIFZMREwgeSB0cmlnbGljw6lyaWRvczogZW50cmUgbm9ybWFsIHkgYWx0b1xuXG5Fc3RvcyB2YWxvcmVzIGxpZ2VyYW1lbnRlIGVsZXZhZG9zIG5vIHNvbiBtb3Rpdm8gZGUgcHJlb2N1cGFjacOzbiBpbm1lZGlhdGEsIHBlcm8gZXMgaW1wb3J0YW50ZSBzZWd1aXIgdW4gZXN0aWxvIGRlIHZpZGEgc2FsdWRhYmxlIHBhcmEgbWFudGVuZXIgbG9zIG5pdmVsZXMgZGUgbMOtcGlkb3MgYmFqbyBjb250cm9sLiBFc3RvIGluY2x1eWUgdW5hIGRpZXRhIHNhbHVkYWJsZSwgZWplcmNpY2lvIHJlZ3VsYXIgeSBubyBmdW1hci5cblxuKipPdHJvcyByZXN1bHRhZG9zOioqXG5cbiogKipVcm9hbsOhbGlzaXM6KipcbiAgICAqIExldWNvY2l0b3MgZW4gZWwgc2VkaW1lbnRvIHVyaW5hcmlvOiBsaWdlcmFtZW50ZSBlbGV2YWRvcyAoNSw5NCBDZWxscy/CtUwpXG4gICAgKiBNb2NvIGVuIGVsIHNlZGltZW50byB1cmluYXJpbzogcG9zaXRpdm9cblxuTGEgcHJlc2VuY2lhIGRlIGxldWNvY2l0b3MgeSBtb2NvIGVuIGVsIHNlZGltZW50byB1cmluYXJpbyBwdWVkZSBpbmRpY2FyIHVuYSBpbmZlY2Npw7NuIG8gaW5mbGFtYWNpw7NuIGRlbCB0cmFjdG8gdXJpbmFyaW8uIEVzIHJlY29tZW5kYWJsZSBjb25zdWx0YXIgY29uIHVuIG3DqWRpY28gcGFyYSBkZXNjYXJ0YXIgY3VhbHF1aWVyIHByb2JsZW1hIHN1YnlhY2VudGUuXG5cbioqRW4gZ2VuZXJhbCwqKiBsb3MgcmVzdWx0YWRvcyBkZSBsYXMgcHJ1ZWJhcyBzb24gKipidWVub3MqKiwgZXhjZXB0byBwb3IgbG9zIG5pdmVsZXMgbGlnZXJhbWVudGUgZWxldmFkb3MgZGUgY29sZXN0ZXJvbCBWTERMIHkgdHJpZ2xpY8Opcmlkb3MsIHkgbGEgcHJlc2VuY2lhIGRlIGxldWNvY2l0b3MgeSBtb2NvIGVuIGVsIHNlZGltZW50byB1cmluYXJpby4gRXMgaW1wb3J0YW50ZSBzZWd1aXIgdW4gZXN0aWxvIGRlIHZpZGEgc2FsdWRhYmxlIHkgY29uc3VsdGFyIGNvbiB1biBtw6lkaWNvIHBhcmEgYWJvcmRhciBlc3RvcyBwcm9ibGVtYXMgbWVub3Jlcy4ifV0=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MC45LCJ0b3BfcCI6MSwidG9wX2siOjAsIm1heF9vdXRwdXRfdG9rZW5zIjoyMDQ4LCJzdG9wX3NlcXVlbmNlcyI6W119' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfSEFURV9TUEVFQ0giLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfU0VYVUFMTFlfRVhQTElDSVQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfREFOR0VST1VTX0NPTlRFTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn1d' # @param {isTemplate: true}
user_input_b64 = '' # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))
user_input = base64.b64decode(user_input_b64).decode()
stream = False

In [ ]:
contents

[{'role': 'user', 'parts': [{'text': 'hello'}]},
 {'role': 'model', 'parts': [{'text': 'Hello! How can I assist you today?'}]}]

In [ ]:
generation_config

{}

In [ ]:
safety_settings

{}

In [ ]:
user_input

'How does electricity work?'

### Call the API

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

chat = gemini.start_chat(history=contents)

response = chat.send_message(
    user_input,
    stream=stream)

In [ ]:
display(Markdown(response.text))

Electricity is the flow of electric charge. It is a fundamental part of nature and is responsible for a wide range of phenomena, from lightning and static electricity to the operation of electronic devices.

**Basic Principles of Electricity:**

1. **Electric Charge:** Matter is made up of atoms, which have a positively charged nucleus surrounded by negatively charged electrons. The movement of these charged particles creates electric currents.

2. **Electric Field:** Electric charges create an electric field around them. The strength and direction of this field depend on the amount and type of charge.

3. **Electric Potential:** Electric potential is the amount of electrical energy stored in a region of space. It is measured in volts (V).

4. **Electric Current:** Electric current is the flow of electric charge through a conductor. It is measured in amperes (A).

**How Electricity Works:**

1. **Generation of Electricity:** Electricity can be generated in various ways, including:
   - **Mechanical Energy:** Generators convert mechanical energy into electrical energy by spinning a conductor in a magnetic field. This is the principle behind power plants that use turbines driven by steam, water, or wind.
   - **Chemical Energy:** Batteries generate electricity through chemical reactions.
   - **Solar Energy:** Solar cells convert sunlight directly into electricity using the photovoltaic effect.

2. **Transmission of Electricity:** Electricity is transmitted over long distances through power lines. These lines are made of conductive materials, such as copper or aluminum, which allow electrons to flow easily.

3. **Distribution of Electricity:** Once electricity reaches populated areas, it is distributed to homes, businesses, and other consumers through a network of local power lines and transformers.

4. **Utilization of Electricity:** Electricity is used to power various devices and appliances by converting electrical energy into other forms of energy, such as light, heat, or motion.

The flow of electricity in a circuit is driven by the difference in electric potential between two points. When a complete circuit is formed, electrons can flow from a region of higher potential to a region of lower potential, creating an electric current.

Electricity is a versatile form of energy that has revolutionized modern society. It is used in a wide range of applications, from lighting and heating to powering computers and transportation systems.

In [ ]:
response.prompt_feedback

safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}

In [ ]:
response.candidates

[index: 0
content {
  parts {
    text: "Electricity is the flow of electric charge. It is a fundamental part of nature and is responsible for a wide range of phenomena, from lightning and static electricity to the operation of electronic devices.\n\n**Basic Principles of Electricity:**\n\n1. **Electric Charge:** Matter is made up of atoms, which have a positively charged nucleus surrounded by negatively charged electrons. The movement of these charged particles creates electric currents.\n\n2. **Electric Field:** Electric charges create an electric field around them. The strength and direction of this field depend on the amount and type of charge.\n\n3. **Electric Potential:** Electric potential is the amount of electrical energy stored in a region of space. It is measured in volts (V).\n\n4. **Electric Current:** Electric current is the flow of electric charge through a conductor. It is measured in amperes (A).\n\n**How Electricity Works:**\n\n1. **Generation of Electricity:** Electr